<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключение-к-БД-и-изучение-таблиц." data-toc-modified-id="Подключение-к-БД-и-изучение-таблиц.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключение к БД и изучение таблиц.</a></span></li><li><span><a href="#Решение-задач." data-toc-modified-id="Решение-задач.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Решение задач.</a></span></li><li><span><a href="#Вывод." data-toc-modified-id="Вывод.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод.</a></span></li></ul></div>

# Проект по SQL.

**Опиcание проекта.**  
 
Ваша компания купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.  
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.  

**Задачи проекта.**  
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.  

**Описание данных.**  

**Таблица `books`**

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.  

**Таблица `authors`**

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.   

**Таблица `publishers`**

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;   

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.   

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

## Подключение к БД и изучение таблиц.

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Создадим подключение к базе данных.

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Исследуем информацию о таблицах, хранящихся в базе данных.

In [3]:
def get_table(name):
    ''' Функция get_table на вход принимает название таблицы, хранящейся в базе данных.
    С помощью запроса к БД формирует и выдает датафрейм
    '''
    
    query = ''' SELECT * FROM {}; 
    '''.format(name)
    request = pd.io.sql.read_sql(query, con = engine)
    return request

In [4]:
books = get_table('books')
authors = get_table('authors')
publishers = get_table('publishers')
ratings = get_table('ratings')
reviews = get_table('reviews')

Изучим информацию о таблицах.

In [5]:
def df_info(df):
    '''Функция возвращает краткую информацию о датафрейме.
    '''
    
    display(df.head(5))
    print()
    print(df.info())

In [6]:
df_info(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


In [7]:
books.book_id.nunique()

1000

В таблице с данными о книгах пропусков нет. Содержится информация о 1000 уникальных книг. Тип данных в столбце с датой публикации книги - строковый.

In [8]:
df_info(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None


In [9]:
authors.author_id.nunique()

636

В таблице с данными об авторах пропусков нет. Содержится информация о 636 уникальных авторах.

In [10]:
df_info(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


In [11]:
publishers.publisher_id.nunique()

340

В таблице с данными об издательствах пропусков нет. Все 340 строк датафрейма это уникальные издательства.

In [12]:
df_info(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


In [13]:
ratings.rating_id.nunique()

6456

In [14]:
ratings.rating.describe()

count    6456.000000
mean        3.928284
std         0.943303
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

В таблице с данными о пользовательских оценках книг пропусков нет. Читателями оставлено 6456 уникальных оценок. Оценивание происходит по 5-ти бальной шкале. Средняя оценка по имеющимся книгам составляет 3,93 балла.

In [15]:
df_info(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None


In [16]:
reviews.review_id.nunique()

2793

В таблице с данными о пользовательских обзорах на книги пропусков нет. Читателями оставлено 2793 уникальных обзора на книги.

## Решение задач.

**Задача №1**  

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [17]:
query_1 = ''' 
SELECT COUNT(*) AS books_count
FROM books
WHERE publication_date > '2000-1-1'; 
''' 

request_1 = pd.io.sql.read_sql(query_1, con = engine)
request_1

,books_count
0,819


**Задача №2**   

Посчитаем для каждой книги количество обзоров и среднюю оценку.

In [18]:
query_2 = ''' 
SELECT 
    b.title AS book_name,
    b.book_id AS book_id,
    rev.cnt AS reviews_count,
    rat.avg_rat AS avg_rating
FROM books AS b 
LEFT JOIN 
         (SELECT book_id,
                 AVG(rating) AS avg_rat
          FROM ratings
          GROUP BY book_id
         ) AS rat ON b.book_id = rat.book_id
LEFT JOIN
         (SELECT book_id,
                 COUNT(review_id) AS cnt
          FROM reviews
          GROUP BY book_id
          ) AS rev ON b.book_id = rev.book_id; 
''' 

request_2 = pd.io.sql.read_sql(query_2, con = engine)
request_2

,book_name,book_id,reviews_count,avg_rating
0,The Body in the Library (Miss Marple #3),652,2.0,4.500000
1,Galápagos,273,2.0,4.500000
2,A Tree Grows in Brooklyn,51,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,951,2.0,4.000000
4,The Prophet,839,4.0,4.285714
...,...,...,...,...
995,Alice in Wonderland,64,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),55,2.0,5.000000
997,Christine,148,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),790,2.0,3.500000


**Задача №3**   

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.

In [19]:
query_3 = ''' 
SELECT i.publisher_id,
       i.publisher,
       i.books_count
FROM
    (SELECT b.publisher_id AS publisher_id,
            COUNT(b.book_id) AS books_count,
            p.publisher AS publisher 
     FROM books AS b
          LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
     WHERE num_pages > 50
     GROUP BY b.publisher_id,
              p.publisher
     ORDER BY books_count DESC) AS i
LIMIT 1;
''' 

request_3 = pd.io.sql.read_sql(query_3, con = engine)
request_3

,publisher_id,publisher,books_count
0,212,Penguin Books,42


**Задача №4**   

Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками.

In [23]:
query_4 = ''' 
SELECT i.author,
       AVG(i.avg_rating) AS avg_rating
FROM
     (SELECT a.author_id AS author_id,
             a.author AS author, 
             AVG(r.rating) AS avg_rating
      FROM books AS b
      LEFT JOIN authors AS a ON b.author_id = a.author_id
      LEFT JOIN ratings AS r ON b.book_id = r.book_id
      GROUP BY b.book_id,
               a.author_id
      HAVING COUNT(r.rating_id) >=50 
     ) AS i 
GROUP BY i.author
ORDER BY avg_rating DESC
LIMIT 1;
''' 

request_4 = pd.io.sql.read_sql(query_4, con = engine)
request_4

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


**Задача №5**  

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [21]:
query_5 = ''' 
SELECT AVG(r.reviews_count) AS avg_rev_count
FROM
    (SELECT username,
            COUNT(review_id) AS reviews_count
     FROM reviews
     WHERE username IN (SELECT username
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating_id) > 50
                       )
     GROUP BY username
    ) AS r; 
''' 

request_5 = pd.io.sql.read_sql(query_5, con = engine)
request_5

,avg_rev_count
0,24.333333


## Вывод.

Выполнили подключение к базе данных сервиса для чтения книг и изучили общую информацию о таблицах, входящих в неё.  
Решили поставленные задачи с помощью SQL-запросов к базе данных и получили следующие ответы:
- После 1 января 2000 г. вышло 819 книг;
- Выгрузили таблицу, в которой для каждой книги посчитано количество обзоров и средняя оценка по ним;
- Издательство "Penguin Books" выпустило 42 книги - это наибольшее число книг толще 50 страниц;
- Самая высокая средняя оценка книг (учитывая только книги с 50 и более оценками) равна 4,28 балла и принадлежит авторам J.K. Rowling / Mary GrandPré;
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, равно 24.